In [3]:

#imports
from tkinter import *
import os
import re
import time
import datetime

import telnetlib
import getpass 
import sys
import time
import serial
import threading #potentially replaced by multiprocessing
from multiprocessing.pool import ThreadPool

import pandas as pd
import numpy as np

from pdfrw import PdfReader
import matplotlib.pyplot as plt
import PyPDF2
from tkinter.filedialog import askopenfilename
os.environ.setdefault('MAGICK_HOME', '/usr/local/Cellar/imagemagick@6/6.9.9-34/')
from wand.image import Image
from PIL import Image as PI
import pyocr
import pyocr.builders
import io


#global defines
nport_ip = "10.136.1.99"
ref_port = "4001"

In [37]:
def get_cals_37():
    global nport_ip
    rma_telnet = telnetlib.Telnet(nport_ip, "4002")
    rma_telnet.write("id?\r\n".encode("ascii"))
    time.sleep(1)
    rma_telnet.write("id?\r\n".encode("ascii"))
    time.sleep(1)
    sbe_id = str(rma_telnet.read_very_eager())[5:7]
    rma_telnet.write(("#"+sbe_id+"DC\r\n").encode("ascii"))
    time.sleep(1)
    #print(rma_telnet.read_very_eager()) #cal data to be parsed
    caldata = rma_telnet.read_very_eager()
    rma_telnet.close()
    return caldata.decode('unicode-escape')



def compare_cals(instr, sheet):
    i_s = instr
    s_s = sheet
    comparison_array = []
    instrument = parse_cal_tags(i_s)
    sheet_return = parse_sheets(s_s)     
    for i in range(len(instrument[0])):
        try:
            comparison_array.append((instrument[0][i], instrument[1][i], sheet_return[i]))
        except:
            pass
    return comparison_array

def parse_cal_tags(cal_string):
    m = re.findall(r"(?<=\s)(.*)(?=\s)", cal_string)
    calibration_tags = []
    calibration_rows = []
    for x in m:
        entry = x.split('=')
        try:
            if len(entry) >= 2:
                calibration_tags.append(entry[0].strip())
                calibration_rows.append(entry[1].strip())
        except:
            pass
    return (calibration_tags, calibration_rows)


def split_sheet(sheet):
    try:
        m = sheet[0].split(' ')[0]
        return m
    except:
        pass

def parse_sheets(s_s):    
    sheet_return = []
    instrument = parse_cal_tags(instr)
    for i in range(len(instrument[0])):
        tags = instrument[0]
        sheet_regex = r"(?i)(?<=\s" + instrument[0][i] + " = )(.*)(?=\s)"
        m = re.findall(sheet_regex, s_s)
        sheet_return.append(split_sheet(m))
    return sheet_return

instr_test = "b'SBE37SI-RS485 V 3.2  5679\\r\\ntemperature:  25-Mar-17\\r\\n    TA0 = 4.261456e-05\\r\\n    TA1 = 2.731360e-04\\r\\n    TA2 = -2.206636e-06\\r\\n    TA3 = 1.490857e-07\\r\\nconductivity:  25-Mar-17\\r\\n    G = -9.994807e-01\\r\\n    H = 1.566288e-01\\r\\n    I = -3.317297e-04\\r\\n    J = 4.905268e-05\\r\\n    CPCOR = -9.570000e-08\\r\\n    CTCOR = 3.250000e-06\\r\\n    WBOTC = 1.185617e-06\\r\\nreference pressure = 2007.0 decibars\\r\\n<Executed/>\\r\\n'"
sheet_test = open("Output.txt", 'r').read()
instr = get_cals_37().decode('unicode-escape')

#parse_sheets(sheet)
#sheet
df = pd.DataFrame(data = compare_cals(instr, sheet_test), columns=['Tag', 'Ins Val', 'Sheet Val'])#columns=['Value', 'Instrument', 'Sheet'])
# print(df)
#parse_cal_tags(instr)
parse_sheets(sheet_test)

AttributeError: 'str' object has no attribute 'decode'

In [36]:
parse_sheets(sheet_test)

0
1
2
3
4
5
6
7
8
9
10
11


[None,
 None,
 None,
 None,
 '—1.0l7709e+000',
 '1.328397e—001',
 '—l.413619e—004',
 None,
 '~9.5700e—008',
 '3.2500e—006',
 None,
 None]

In [ ]:
#test of sample collection --> to collect 10 samples

def connect_ref(s):
    tn = s
    sample_time = datetime.datetime.now().strftime('%Y/%m/%d %H:%M:%S:%f')
    tn.write(("TS\r\n").encode('ascii'))
    time.sleep(2)
    return data
    
def connect_rma(s):
    tn = s
    sample_time = datetime.datetime.now().strftime('%Y/%m/%d %H:%M:%S:%f')
    tn.write(("#03TS\r\n").encode('ascii'))
    time.sleep(2)
    return data

def clean_sample(dirty_sample):
    trantab = str.maketrans('', '', 'b\'\\rn<Executed/>TS=i Bpls')
    translate = dirty_sample.translate(trantab)
    return translate

def sample_todf(ref_sample, rma_sample, rma_port):
    ref_sample = ref_sample.split(',')[:4]
    rma = rma_sample.split(',')
    if rma_port == "4002": #correct this for ins type - potentially need a check for pressure
        rma_sample = [rma[0], rma[3], rma[4], rma[5]]
    if rma_port == "4003":     #correct this for ins type
        rma_sample = rma[:3]
    df = pd.DataFrame(data=[ref_sample, rma_sample], columns=['datetime', 'temperature', 'conductivity', 'pressure'])
    return df
      
def collect_samples(rma_port):
    global nport_ip
    ref_sample = []
    rma_sample = []
    rma_tn = telnetlib.Telnet(nport_ip, rma_port, 5)
    ref_tn = telnetlib.Telnet(nport_ip, "4001", 5) 
    
    ref_tn.write(("\r\n").encode('ascii'))
    time.sleep(1)
    sample_time = datetime.datetime.now().strftime('%Y/%m/%d %H:%M:%S:%f')
    ref_tn.write(("TS\r\n").encode('ascii'))
    
    rma_tn.write(("id?\r\n").encode('ascii'))
    time.sleep(1)
    sample_time = datetime.datetime.now().strftime('%Y/%m/%d %H:%M:%S:%f')
    rma_tn.write(("#03TS\r\n").encode('ascii'))
    
    for i in range(10):
            pool = ThreadPool(processes=2)
            procRMA = pool.apply_async(connect_rma, args=(rma_tn,))
            procREF = pool.apply_async(connect_ref, args=(ref_tn,))
            
#   rma_sample = clean_sample(rma_sample)
#   ref_sample = clean_sample(ref_sample)
    rma_tn.close()
    ref_tn.close()
    return sample_todf(ref_sample, rma_sample, rma_port)

collect_samples("4002")

In [72]:
hi = "hihi"
instr_data = (['TA0',
      'TA1',
      'TA2',
      'TA3',
      'G',
      'H',
      'I',
      'J',
      'CPCOR',
      'CTCOR',
      'WBOTC',
      'reference pressure'],
     ['4.261456e-05',
      '2.731360e-04',
      '-2.206636e-06',
      '1.490857e-07',
      '-9.994807e-01',
      '1.566288e-01',
      '-3.317297e-04',
      '4.905268e-05',
      '-9.570000e-08',
      '3.250000e-06',
      '1.185617e-06',
      '2007.0 decibars'])
root = Tk()




In [38]:
def gen_report():
    def stringvar_list(lst):
        vals = [str(i) for i in lst]
        ret_lst = [None]*len(lst)
        for i in range(len(lst)):
            ret_lst[i] = StringVar()
            ret_lst[i].set(vals[i])
        return ret_lst
    def update_cal_var():
        for i in range(length):
            try:
                sheet_values[i].set(update_val[i].get())
                print("stringvar: ", update_val[i])
            except Exception as e:
                print (e)
    report = Tk()
    report.geometry("1000x600")
    report.title("Report")
    
    cals = pd.DataFrame(data =[[1,2,3], [1,2,3], [1,2,3]], columns=['a','b','c'])
    samples = pd.DataFrame(data =[[1,2,3], [1,2,3], [1,2,3]], columns=['a','b','c'])
    instr_data = (['TA0',
      'TA1',
      'TA2',
      'TA3',
      'G',
      'H',
      'I',
      'J',
      'CPCOR',
      'CTCOR',
      'WBOTC',
      'reference pressure'],
     ['4.261456e-05',
      '2.731360e-04',
      '-2.206636e-06',
      '1.490857e-07',
      '-9.994807e-01',
      '1.566288e-01',
      '-3.317297e-04',
      '4.905268e-05',
      '-9.570000e-08',
      '3.250000e-06',
      '1.185617e-06',
      '2007.0 decibars'])

    sheet_data = [None,
     None,
     None,
     None,
     '—1.0l7709e+000',
     '1.328397e—001',
     '—l.413619e—004',
     None,
     '~9.5700e—008',
     '3.2500e—006',
     None,
     None]

    sheet_data = ['None' if v is None else v for v in sheet_data]

    instr_tags = stringvar_list(instr_data[0])
    instr_values = stringvar_list(instr_data[1])
    sheet_values = stringvar_list(sheet_data)
    
        
    length = len(instr_data[0])
    tags = [None] * length
    ins_val = [None] * length
    she_val = [None] * length
    update_entry = [None] * length
    update_val = [None]*length
    
    cal_label = Label(report, text="Calibration Comparison", height=1, width=20)
    cal_label.grid(sticky='NW', columnspan=3)
          
    for i in range(length):
        tags[i] = Text(report, height=1, width=15)
        tags[i].insert(INSERT, instr_tags[i].get())
        tags[i].grid(column=0, row=i+1)
        ins_val[i] = Text(report, height=1, width=15)
        ins_val[i].insert(INSERT, instr_values[i].get())
        ins_val[i].grid(column=1, row=i+1, sticky='NW')
        she_val[i] = Label(report, height=1, width=15, text=sheet_values[i].get() , textvariable=sheet_values[i])
        she_val[i].grid(column=2, row=i+1, sticky='NW', padx = 10)
        update_val[i] = StringVar()
        update_entry[i] = Entry(report ,textvariable = update_val[i])
        update_entry[i].insert(END, sheet_values[i].get())
        update_entry[i].grid(column=3, row=i+1, sticky='NW')
    
    update_button = Button(report, text='Update Values', command=update_cal_var)
    update_button.grid(row=length+1, columnspan=3)
        
    mainloop()
    
gen_report()

stringvar:  PY_VAR785
stringvar:  PY_VAR786
stringvar:  PY_VAR787
stringvar:  PY_VAR788
stringvar:  PY_VAR789
stringvar:  PY_VAR790
stringvar:  PY_VAR791
stringvar:  PY_VAR792
stringvar:  PY_VAR793
stringvar:  PY_VAR794
stringvar:  PY_VAR795
stringvar:  PY_VAR796
stringvar:  PY_VAR785
stringvar:  PY_VAR786
stringvar:  PY_VAR787
stringvar:  PY_VAR788
stringvar:  PY_VAR789
stringvar:  PY_VAR790
stringvar:  PY_VAR791
stringvar:  PY_VAR792
stringvar:  PY_VAR793
stringvar:  PY_VAR794
stringvar:  PY_VAR795
stringvar:  PY_VAR796


In [108]:
def stringvar_list(lst):
    #rooster = Tk()
    vals = [str(i) for i in lst]
    ret_lst = [None]*len(lst)
    for i in range(len(lst)):
        ret_lst[i] = StringVar()
        ret_lst[i].set(vals[i])
    return ret_lst

instr_data = (['TA0',
  'TA1',
  'TA2',
  'TA3',
  'G',
  'H',
  'I',
  'J',
  'CPCOR',
  'CTCOR',
  'WBOTC',
  'reference pressure'],
 ['4.261456e-05',
  '2.731360e-04',
  '-2.206636e-06',
  '1.490857e-07',
  '-9.994807e-01',
  '1.566288e-01',
  '-3.317297e-04',
  '4.905268e-05',
  '-9.570000e-08',
  '3.250000e-06',
  '1.185617e-06',
  '2007.0 decibars'])

outs = stringvar_list(instr_data[1])
outs[1].get()

'2.731360e-04'

In [31]:
a = [1]*10
b = [2]*10
c = (a, b)
c[0]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [104]:
a = Tk()
b = StringVar()
b.set("hi")
b.get()

'hi'